In [19]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from alpaca_trade_api import TimeFrame
import datetime
from dateutil.relativedelta import relativedelta

%matplotlib inline


In [20]:
load_dotenv()

# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('Alpaca_API_Key')
alpaca_secret_key = os.getenv('Alpaca_Secret_Key')

if type(alpaca_api_key) ==type(''):
    if type(alpaca_secret_key) ==type(''):
        print('You have both Alpaca Keys saved in your environment variables')
    else:
        print('You have the Alpaca Key but no Secret Key saved in your environment variables')
else:
    print('You do not have any Alpaca API Keys saved in your environment variables')

You have both Alpaca Keys saved in your environment variables


In [21]:
# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='V2')

In [41]:
# This is where the user inputs are collected and stored as variables

# Set the tickers for both the bond and stock portion of the portfolio
portfolio_tickers = ['GME','AAPL','MSFT','AMZN','NVDA','AMD','AMC','GOOG','FB','XOM']

comparison_tickers = ['SPY','AGG']

# Set the number of years history you'd like the API to pull for the analysis
number_of_years_history = 4

In [42]:
# Set timeframe to 1D 
timeframe = TimeFrame.Day


number_of_trading_days = 252.5 * number_of_years_history
number_of_tickers = len(portfolio_tickers)
limit = number_of_trading_days*number_of_tickers

limit_rows = limit

# Format current date as ISO format
start_date = pd.Timestamp(datetime.date.today() - relativedelta(years=number_of_years_history), tz='America/New_York').isoformat()
end_date = pd.Timestamp(datetime.date.today(), tz='America/New_York').isoformat()

print(f"The approximate number of rows the 'alpaca_prices_df' will produce is: {limit}")

The approximate number of rows the 'alpaca_prices_df' will produce is: 10100.0


In [43]:
def alpaca_multi_index_df(tickers, tf, start, end, rows):
    stock_prices_multi_index_df = pd.DataFrame()
    count = 0
    for ticker in tickers:
        alpaca_prices = alpaca.get_bars(ticker, tf ,start=start, end=end, limit=rows).df
        column_names = [(ticker, x) for x in alpaca_prices.columns]
        alpaca_prices.columns = pd.MultiIndex.from_tuples(column_names)
        count = count + 1
        if count == 1:
            stock_prices_multi_index_df = alpaca_prices
        else:
            stock_prices_multi_index_df = pd.merge(stock_prices_multi_index_df, alpaca_prices, how = 'outer', left_index=True, right_index=True)
    return stock_prices_multi_index_df


In [44]:
portfolio_multi_index_df = alpaca_multi_index_df(portfolio_tickers, timeframe, start_date, end_date, limit_rows)
portfolio_multi_index_df

GME                                       \
                              open      high      low   close    volume   
timestamp                                                                 
2018-04-03 04:00:00+00:00   12.640   13.1250   12.600   13.01   5747536   
2018-04-04 04:00:00+00:00   12.820   13.4800   12.710   13.34   6383809   
2018-04-05 04:00:00+00:00   13.350   13.6300   13.270   13.56   5140634   
2018-04-06 04:00:00+00:00   13.420   13.6600   13.240   13.43   4238278   
2018-04-09 04:00:00+00:00   13.510   13.7600   13.430   13.48   5703464   
...                            ...       ...      ...     ...       ...   
2022-03-28 04:00:00+00:00  152.590  190.8400  151.545  189.59  16264467   
2022-03-29 04:00:00+00:00  188.620  199.4100  163.000  179.90  18018943   
2022-03-30 04:00:00+00:00  175.025  183.3369  165.000  166.85   9092904   
2022-03-31 04:00:00+00:00  163.100  175.7450  158.510  166.58  11239609   
2022-04-01 04:00:00+00:00  188.690  189.7688  155.260  165.00  13184045   

                                                     AAPL                     \
                          trade_count        vwap    open      high      low   
timestamp                                                                      
2018-04-03 04:00:00+00:00       30285   12.960382  167.58  168.7455  164.880   
2018-04-04 04:00:00+00:00       36400   13.254810  164.90  172.0100  164.770   
2018-04-05 04:00:00+00:00       30219   13.513630  172.53  174.2304  172.080   
2018-04-06 04:00:00+00:00       23219   13.400160  171.02  172.4800  168.200   
2018-04-09 04:00:00+00:00       30938   13.552234  170.06  173.0900  169.845   
...                               ...         ...     ...       ...      ...   
2022-03-28 04:00:00+00:00      303588  170.866471  172.12  175.7300  172.000   
2022-03-29 04:00:00+00:00      349738  181.076872  176.75  179.0100  176.340   
2022-03-30 04:00:00+00:00      175443  174.402192  178.51  179.6100  176.700   
2022-03-31 04:00:00+00:00      199225  172.584853  177.80  178.0300  174.400   
2022-04-01 04:00:00+00:00      277942  173.455744  174.05  174.8800  171.940   

                           ...        FB                            XOM  \
                           ...    volume trade_count        vwap   open   
timestamp                  ...                                            
2018-04-03 04:00:00+00:00  ...  42544313      266330  154.758724  73.46   
2018-04-04 04:00:00+00:00  ...  49896679      290777  153.376673  74.03   
2018-04-05 04:00:00+00:00  ...  41461203      262463  159.263181  75.09   
2018-04-06 04:00:00+00:00  ...  41649237      269130  158.818622  75.59   
2018-04-09 04:00:00+00:00  ...  34926931      214574  158.956174  75.22   
...                        ...       ...         ...         ...    ...   
2022-03-28 04:00:00+00:00  ...  26224491      339795  222.200529  83.20   
2022-03-29 04:00:00+00:00  ...  31417945      408695  228.397803  80.41   
2022-03-30 04:00:00+00:00  ...  25588046      289815  228.774735  83.16   
2022-03-31 04:00:00+00:00  ...  24124274      307893  224.602652  83.14   
2022-04-01 04:00:00+00:00  ...  19507320      287776  224.611383  82.21   

                                                                       \
                            high     low  close    volume trade_count   
timestamp                                                               
2018-04-03 04:00:00+00:00  75.14  73.100  75.02  16001382       88251   
2018-04-04 04:00:00+00:00  75.06  73.440  74.87  13122670       74124   
2018-04-05 04:00:00+00:00  76.32  75.020  76.02  14203277       74791   
2018-04-06 04:00:00+00:00  76.12  74.215  74.87  14299567       93883   
2018-04-09 04:00:00+00:00  75.79  74.800  74.87  11102619       60859   
...                          ...     ...    ...       ...         ...   
2022-03-28 04:00:00+00:00  83.31  82.020  82.81  21538613      191429   
2022-03-29 04:00:00+00:00  82.49  79.760  82.37  27014308      237374   
2022-

In [45]:
portfolio_prices_df = alpaca.get_bars(portfolio_tickers, timeframe ,start=start_date, end=end_date, limit=limit_rows).df
portfolio_prices_df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2018-04-03 04:00:00+00:00,167.58,168.7455,164.880,168.39,30279470,208593,166.937349,AAPL
2018-04-04 04:00:00+00:00,164.90,172.0100,164.770,171.61,34605489,240829,169.256333,AAPL
2018-04-05 04:00:00+00:00,172.53,174.2304,172.080,172.80,26933298,189685,173.221157,AAPL
2018-04-06 04:00:00+00:00,171.02,172.4800,168.200,168.38,35006481,251952,169.828996,AAPL
2018-04-09 04:00:00+00:00,170.06,173.0900,169.845,170.05,29017712,199534,171.481740,AAPL
...,...,...,...,...,...,...,...,...
2022-03-28 04:00:00+00:00,83.20,83.3100,82.020,82.81,21538613,191429,82.760446,XOM
2022-03-29 04:00:00+00:00,80.41,82.4900,79.760,82.37,27014308,237374,81.153129,XOM
2022-03-30 04:00:00+00:00,83.16,84.0100,82.710,83.78,22069734,193040,83.485671,XOM


In [36]:
comparison_multi_index_df = alpaca_multi_index_df(comparison_tickers, timeframe, start_date, end_date, limit_rows)
comparison_multi_index_df

SPY                                        \
                             open     high       low   close     volume   
timestamp                                                                 
2018-04-03 04:00:00+00:00  258.92  261.310  256.8400  260.77  120373843   
2018-04-04 04:00:00+00:00  256.76  264.355  256.6000  263.56  126181329   
2018-04-05 04:00:00+00:00  265.55  266.640  264.3200  265.64   82783739   
2018-04-06 04:00:00+00:00  263.42  265.110  258.0000  259.72  183222666   
2018-04-09 04:00:00+00:00  261.37  264.840  259.9356  261.00  105506132   
...                           ...      ...       ...     ...        ...   
2022-03-28 04:00:00+00:00  452.05  455.910  450.0600  455.84   68744767   
2022-03-29 04:00:00+00:00  460.07  462.070  457.1800  461.53   86682895   
2022-03-30 04:00:00+00:00  460.29  461.195  456.4650  458.79   79668682   
2022-03-31 04:00:00+00:00  457.92  458.760  451.1600  452.11  118607132   
2022-04-01 04:00:00+00:00  453.30  453.460  449.1400  452.92   88676014   

                                                      AGG                     \
                          trade_count        vwap    open      high      low   
timestamp                                                                      
2018-04-03 04:00:00+00:00      451161  259.007457  106.82  106.8700  106.740   
2018-04-04 04:00:00+00:00      463570  260.914353  106.90  106.9267  106.791   
2018-04-05 04:00:00+00:00      318778  265.452359  106.71  106.7600  106.630   
2018-04-06 04:00:00+00:00      563147  261.303855  106.89  106.9763  106.790   
2018-04-09 04:00:00+00:00      333763  262.165085  106.83  107.0000  106.780   
...                               ...         ...     ...       ...      ...   
2022-03-28 04:00:00+00:00      624225  453.296491  106.25  106.5252  106.160   
2022-03-29 04:00:00+00:00      738526  459.632992  106.61  106.8900  106.455   
2022-03-30 04:00:00+00:00      659199  459.158702  106.60  107.1500  106.590   
2022-03-31 04:00:00+00:00      751169  455.444107  107.19  107.3800  107.060   
2022-04-01 04:00:00+00:00      624745  451.850848  106.35  106.9500  106.250   

                                                                     
                            close    volume trade_count        vwap  
timestamp                                                            
2018-04-03 04:00:00+00:00  106.78   9916091       33232  106.808401  
2018-04-04 04:00:00+00:00  106.82   6200937       17851  106.839944  
2018-04-05 04:00:00+00:00  106.67   2592453       11216  106.679372  
2018-04-06 04:00:00+00:00  106.91   2814781       12014  106.905096  
2018-04-09 04:00:00+00:00  107.00   3746468       14462  106.889641  
...                           ...       ...         ...         ...  
2022-03-28 04:00:00+00:00  106.36   8540551       27246  106.295912  
2022-03-29 04:00:00+00:00  106.86  11396549       33651  106.734051  
2022-03-30 04:00:00+00:00  107.15  11544378       24975  107.007713  
2022-03-31 04:00:00+00:00  107.10  13788220       44817  107.229272  
2022-04-01 04:00:00+00:00  106.75  11579091       40126  106.667931  

[1009 rows x 14 columns]

In [37]:
comparison_prices_df = alpaca.get_bars(comparison_tickers, timeframe ,start=start_date, end=end_date, limit=limit_rows).df
comparison_prices_df

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2018-04-03 04:00:00+00:00,106.82,106.8700,106.740,106.78,9916091,33232,106.808401,AGG
2018-04-04 04:00:00+00:00,106.90,106.9267,106.791,106.82,6200937,17851,106.839944,AGG
2018-04-05 04:00:00+00:00,106.71,106.7600,106.630,106.67,2592453,11216,106.679372,AGG
2018-04-06 04:00:00+00:00,106.89,106.9763,106.790,106.91,2814781,12014,106.905096,AGG
2018-04-09 04:00:00+00:00,106.83,107.0000,106.780,107.00,3746468,14462,106.889641,AGG
...,...,...,...,...,...,...,...,...
2022-03-28 04:00:00+00:00,452.05,455.9100,450.060,455.84,68744767,624225,453.296491,SPY
2022-03-29 04:00:00+00:00,460.07,462.0700,457.180,461.53,86682895,738526,459.632992,SPY
2022-03-30 04:00:00+00:00,460.29,461.1950,456.465,458.79,79668682,659199,459.158702,SPY
